In [1]:
import os, sys
import numpy as np
import psycopg2
from subprocess import call
import subprocess

In [5]:
# call(["convert", "-size", "100x100", "xc:wheat", "canvas_wheat.gif"])
# call(["convert", "-size", "100x100", "xc:red", "canvas_red.gif"])

0

In [2]:
# set context/cwd

stack_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/images/"
channel_folder = "/Users/OGN/1Projects/ZBrain2/app/assets/3-ColorChannels/"

os.chdir(stack_folder)
cwd = os.getcwd()
print "Current working directory %s" % cwd

Current working directory /Users/OGN/1Projects/ZBrain2/app/assets/images


In [3]:
# get stack names

stacks = os.listdir(stack_folder)
stacks = [ stack for stack in stacks if os.path.isdir(stack) ]
print stacks[:8]

['6.7FRhcrtR-Gal4-uasKaede', 'Anti-5HT', 'Anti-Gad67', 'Anti-GlyR', 'Anti-tERK', 'Anti-TH', 'Anti-Zn1', 'Anti-Zn12(Hnk-1)']


In [5]:
# create color channel folders

colors = ['red', 'blue', 'green', 'cyan']

for stack in stacks:
    folder_path = "/Users/OGN/1Projects/ZBrain2/app/assets/3-ColorChannels/%s/" % stack
    try:
        os.mkdir(folder_path)
    except:
        pass
    for color in colors:
        channel_path = folder_path + color
        try:
            os.mkdir(channel_path)
        except:
            pass

In [8]:
# create color channel overlays for each stack

for s, stack in enumerate(stacks[65:]):
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/images/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/3-ColorChannels/%s/" % stack

    os.chdir(src)
    print s, base
    for color in colors:
        dest = base + color
        #call(["mogrify", "-path", dest, "+level-colors", "black,%s"%color, "-fuzz", "9%", "-transparent", "black", "*.png"]
        cwd = os.getcwd()
        print cwd
        try:
            subprocess.check_output(["mogrify", "-path", dest, "-format", "png", "+level-colors", "black,%s"%color, "-fuzz", "9%", "-transparent", "black", "*.jpg"], stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            print err.output


0 /Users/OGN/1Projects/ZBrain2/app/assets/3-ColorChannels/ZBB_y293-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y293-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y293-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y293-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y293-Gal4
1 /Users/OGN/1Projects/ZBrain2/app/assets/3-ColorChannels/ZBB_y294-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y294-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y294-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y294-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y294-Gal4
2 /Users/OGN/1Projects/ZBrain2/app/assets/3-ColorChannels/ZBB_y295-Gal4/
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y295-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y295-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y295-Gal4
/Users/OGN/1Projects/ZBrain2/app/assets/images/ZBB_y295-Gal4
3 /Users/OGN/1Projects/ZBrain2/app/assets/3-Color

In [17]:
# prepare postgreSQL table

create_table_command = (
"""
    DROP TABLE colorchannels;
    CREATE TABLE colorChannels (
        channel_id SERIAL PRIMARY KEY,
        channel_color TEXT NOT NULL,
        channel_name TEXT NOT NULL,
        channel_image_path TEXT NOT NULL
    );
""")

In [27]:
# connect to zbrain2db postgreSQL database

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [28]:
# create table colorChannels

try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print error
finally:
    if conn is not None:
        conn.close()

In [29]:
from natsort import natsorted

In [30]:
# create array of table insert commands for each color channel image

channel_id = 0
insert_commands = []
for stack in stacks[3:]:
    src = "/Users/OGN/1Projects/ZBrain2/app/assets/imagesbk/%s/" % stack
    base = "/Users/OGN/1Projects/ZBrain2/app/assets/images/3-ColorChannels/%s/" % stack
    rel = "images/3-ColorChannels/%s/" % stack
    
    colors = ['red', 'blue', 'green', 'cyan']
    for color in colors:
        channel_folder = base + color
        channels = os.listdir(channel_folder)
        channels = [ "%s%s/%s" % (rel, color, channel) for channel in channels]
        channels = natsorted(channels)
        for channel_image_path in channels:
            insert_command = ("INSERT INTO colorChannels (channel_id, channel_color, channel_name, channel_image_path) values (%i, '%s', '%s', '%s')" 
                                 % (channel_id, color, stack, channel_image_path))
            channel_id += 1
            insert_commands.append(insert_command)

In [31]:
# use insert command array to create table entries for each color channel image

conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception, e:
        print repr(e)
        
cursor.close()
conn.commit()

In [32]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM colorChannels ORDER BY channel_id')

rows = cursor.fetchall()
for row in rows[300:500]:
    print row

(300, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-24.png')
(301, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-25.png')
(302, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-26.png')
(303, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-27.png')
(304, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-28.png')
(305, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7FRhcrtR-Gal4-uasKaede/green/6.7FRhcrtR-Gal4-uasKaede_6dpf_MeanImageOf12Fish-29.png')
(306, 'green', '6.7FRhcrtR-Gal4-uasKaede', 'images/3-ColorChannels/6.7